In [1]:
using ForwardDiff
using LinearAlgebra
using Plots
using NLsolve

┌ Info: Recompiling stale cache file /Users/pooyarashidi-ravari/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184
┌ Warning: Package Plots does not have FFMPEG in its dependencies:
│ - If you have Plots checked out for development and have
│   added FFMPEG as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Plots
└ Loading FFMPEG into Plots from project dependency, future warnings for Plots are suppressed.
┌ Warning: Package StatsBase does not have DataAPI in its dependencies:
│ - If you have StatsBase checked out for development and have
│   added DataAPI as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with StatsBase
└ Loading DataAPI into StatsBase from project dependency, future warnings for StatsBase are suppressed.
┌ Warni

┌ Warning: The call to compilecache failed to create a usable precompiled cache file for DiffEqDiffTools [01453d9d-ee7c-5054-8395-0335cb756afa]
│   exception = Required dependency ArrayInterface [4fba245c-0d91-5ea0-9b3e-6abc04ee57a9] failed to load from a cache file.
└ @ Base loading.jl:969
┌ Warning: Package DiffEqDiffTools does not have ArrayInterface in its dependencies:
│ - If you have DiffEqDiffTools checked out for development and have
│   added ArrayInterface as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with DiffEqDiffTools
└ Loading ArrayInterface into DiffEqDiffTools from project dependency, future warnings for DiffEqDiffTools are suppressed.
┌ Warning: Package ArrayInterface does not have LinearAlgebra in its dependencies:
│ - If you have ArrayInterface checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environm

In [3]:
function interiorpoint_eqaulity_constraint_method_2(f, x0, eq_c, c;
                       L   = (x,λ)->(f(x) - dot(λ,c(x))),
                       ∇ₓL = (x,λ)->ForwardDiff.gradient(z->L(z,λ), x),
                       ∇²ₓL= (x,λ)->ForwardDiff.hessian(z->L(z,λ), x),
                       ∇c = x->ForwardDiff.jacobian(c,x),
                       tol=1e-4, maxiter = 1000,
                       μ0 = 1.0, μfactor = 0.2,
                       xrange=[-2., 3.],
                       yrange=[-2.,6.], animate=true, verbosity=0)
    #x0 = eq_c(x0, c)
    fold = f(x0)
    xold = x0
    all(c(x0).>=0) || error("interiorpoint requires a starting value that strictly satisfies all constraints")
    μ = μ0
    λ = μ./c(x0)
    xchange=Inf
    fchange=Inf
    iter = 0
    μiter = 0
    stuck=0

    animate = animate && length(x0)==2
    if animate
        # make a contour plot of the function we're minimizing. This is for
        # illustrating; you wouldn't have this normally
        ct = contour(range(xrange[1],xrange[2], length=100), 
                    range(yrange[1],yrange[2], length=100),
                     (x,y) -> log(f([x,y])))
        plot!(ct, xrange, 2.5 .- xrange) # add constraint 
        anim = Animation()
    end
    foc = [∇ₓL(xold,λ); λ.*c(xold)]
    while(iter < maxiter && ((xchange>tol) || (fchange>tol) || (stuck>0)
                           || norm(foc)>tol || μ>tol) )
        # Calculate the direction for updating x and λ
        Dc = ∇c(xold)
        cx = c(xold)
        foc = ∇ₓL(xold, λ)
        H = ∇²ₓL(xold,λ)
        Δ = [H   -Dc'; λ'*Dc  diagm(0=>cx)] \ [-foc; μ .- cx.*λ]

        # Find a step size such that λ>=0 and c(x)>=0
        # The details here could surely be improved
        α = 1.0
        acceptedstep = false
        λold = copy(λ)
        x = copy(xold)
        while (α > 1e-10)
            x = xold + α*Δ[1:length(xold)]
            λ = λold + α*Δ[(length(xold)+1):length(Δ)]
            x_1 = eq_c(x, c)[1]
            x_2 = eq_c(x, c)[2]
            #--------------------------------------------------------------------
            #THIS IS WHERE I CHANGED---------------------------------------------
            #--------------------------------------------------------------------
            if ( (all(λ.>=0) && all(c([x_1, x[2]]).>=0)) || (all(λ.>=0) && all(c([x[1], x_2] ).>=0)) )
                acceptedstep=true
                break
            end
            α *= 0.5
        end
        if !acceptedstep
          stuck = 1
          break
        end
        fnew = f(x)

        if (animate)
          scatter!(ct, [xold[1]],[xold[2]], markercolor=:red, legend=false,
                   xlims=xrange, ylims=yrange) 
          quiver!(ct, [xold[1]],[xold[2]], quiver=([α*Δ[1]],[α*Δ[2]]), legend=false,
                  xlims=xrange, ylims=yrange)
          frame(anim)
        end

        xchange = norm(x-xold)
        fchange = abs(fnew-fold)
        μiter += 1

        # update μ (the details here could also be improved)    
        foc = ∇ₓL(x,λ)
        if (μiter>10 || (norm(foc)< μ && λ'*c(x)<10*μ)) 
          μ *=  μfactor
          μiter = 0
        end

        xold = x
        fold = fnew
        if verbosity>0
          print("Iter $iter: f=$fnew, λ=$λ, c(x)=$(c(x)), μ=$μ, norm(foc)=$(norm(foc))\n")
        end
        iter += 1    
    end
    if (iter >= maxiter)
        info = "Maximum iterations reached"
    elseif (stuck>0)
        info = "Failed to find feasible step for " * string(stuck) * " iterations."
    else
        info = "Convergence."
    end
    return(fold, xold, iter, info, anim) 
end

interiorpoint_eqaulity_constraint_method_2 (generic function with 1 method)

In [4]:
function equality_constraint(x, constraint)
    x_1 = copy([x[1]])
    function e_c_1!(F, x_1)
        F[1] = constraint([x_1[1], x[2]])[1]
    end
    x_1 = nlsolve(e_c_1!, [1.]).zero[1]
    #-------
    x_2 = copy([x[2]])
    function e_c_2!(F, x_2)
        F[1] = constraint([x[1], x_2[1]] )[1]
    end
    x_2 = nlsolve(e_c_2!, [1.]).zero[1]
    return x_1, x_2
end

equality_constraint (generic function with 1 method)

In [5]:
"""
banana(a,b)
  Returns the Rosenbrock function with parameters a, b.
"""
function banana(a,b)
  x->(a-x[1])^2+b*(x[2]-x[1]^2)^2
end
f = banana(1.0,1.0)
x0 = [3.0, 0.0]
function constraint(x)
  [x[1] + x[2] - 2.5]
end

constraint (generic function with 1 method)

In [8]:
result_eq = interiorpoint_eqaulity_constraint_method_2(f, x0,equality_constraint, constraint; maxiter=100)
gif(result_eq[5], "ip.gif", fps=5)

┌ Info: Saved animation to 
│   fn = /Users/pooyarashidi-ravari/Dropbox/UBC/2Y/computational/code/ip.gif
└ @ Plots /Users/pooyarashidi-ravari/.julia/packages/Plots/Iuc9S/src/animation.jl:95


Plots.AnimatedGif("/Users/pooyarashidi-ravari/Dropbox/UBC/2Y/computational/code/ip.gif")

In [7]:
result_eq[2], result_eq[1], constraint(result_eq[2])

([1.14498, 1.35505], 0.022961353831246145, [2.90703e-5])